# Proyecto NLP Quijote

**Objetivo**
Creación de un modelo de Deep Learning entrenado con los primeros **50 capitulos del Quijote**, para crear una contextualización artificial del contenido del libro y de esta manera poder predecir un nuevo texto en función a unas palabras dadas.

**Requisitos**
* Python 3.8
* Tensorflow 2.x

**Pasos de creación**
* Adquirir el libro del Quijote en formato digital. https://www.gutenberg.org/ebooks/search/?query=quijote&submit_search=Go%21
* Tratamiento de los datos
* Creación del modelo usando redes LSTM (Large Short Term Memory)
* Entrenamiento
* Resultados y validación
* Prediccón
* Salvar modelo

In [2]:
#Cargamos las librerias correspondientes
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import re

from codecarbon import EmissionsTracker

### Tokenizer
Como la palabra sugiere tokenizar significa dividir la oración en una serie de tokens o en palabras simples, podemos decir que cada vez que hay un espacio en una oración agregamos una coma entre ellos para que nuestra oración se divida en tokens y cada palabra tenga un valor único de un número entero.

In [3]:
tokenizer = Tokenizer()

data = open('Dataset/quijote_Lite.txt', encoding="utf8").read()
#data = open('/content/drive/My Drive/Colab Notebooks/Datasets/Vuelta_al_mundo.txt').read()

#Limpiar de simbolos 
data = re.sub('[^a-zA-Z0-9á-ú\¿\?\n\.]', ' ', data)
corpus = data.lower().split("\n")

#Mostramos el cuerpo del texto ya limpio
#print(corpus)
corpus = sorted(list(set(corpus)))

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1



### Resultado despues de realizar los **tokens**

In [4]:
#print(tokenizer.word_index)
#print(total_words)
#print(corpus)
#print(len(corpus))

#### Preparación de los datos
Para mejorar el entrenamiento del modelo y poder tener más datos a partir de los obtenidos, se realiza una tecnica llamda **secuencia**. Que consiste en dividir cada oración en una más pequeña en forma de escalera, de esta forma se hará una predicción de entrenamiento por cada subdivisión de esa oración.

<img src="images/secuencias.png">

Posteriormente se realiza un **Padding**. Es un método para convertir la matriz de enteros de longitud variable en una longitud fija, ya sea truncando (si la longitud es mayor que la longitud_máxima definida que trunca la matriz) o rellenando (si la longitud es más corta que la longitud_máxima, rellene la matriz con 0).

<img src="images/Padding.png">

In [5]:
input_sequences = []
#Marcamos los tokens por cada frase
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	#Creamos frases más pequeñas en función a al original
	#for i in range(1, len(token_list)):
	for i in range(1, len(token_list)): #usamos cada frase para aumentar el train en modo de escalera
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
#Buscamos la frase más larga y las igualamos todas con ceros en la parte de adelante
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
#Cogemos el ultimo valor como etiqueta (y) y el resto como (x)
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

**Mostramos algunos ejemplos del proceso**

In [6]:
print(xs[5])
print(ys[5])
print(ys.shape)
print(xs.shape)

[   0    0    0    0    0    0    0    0    0    0    0    0  119 1725
    1 4532    5   25]
[0. 0. 0. ... 0. 0. 0.]
(153835, 13850)
(153835, 18)


### Creación del modelo

* La primera capa incluye el número de palabras a entrenar y la salida de predicción que queremos mostrar, en este caso se hará una predicción de 50 palabras en función a la dada.

* Usamos como optimizador Adam, aunque tambien se optienen buenos resultados con RMSprop

In [7]:
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
#model.add(Bidirectional(LSTM(128)))
#model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.001)
rms=RMSprop(learning_rate=0.01)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
#filepath="/content/drive/My Drive/Colab Notebooks/Projects/QuijoteNLP/model/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
filepath="model/weights-QuijoteNLP.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.summary()



2024-01-15 21:45:37.468869: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-15 21:45:37.470901: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-15 21:45:37.472077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 50)            692500    
                                                                 
 lstm (LSTM)                 (None, 18, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 13850)             1398850   
                                                                 
Total params: 2,242,250
Trainable params: 2,242,250
Non-trainable params: 0
_________________________________________________________________


In [8]:
with EmissionsTracker() as tracker:
    history = model.fit(xs, ys, epochs=10, batch_size=128, verbose=1,callbacks=callbacks_list)

[codecarbon INFO @ 21:45:47] [setup] RAM Tracking...
[codecarbon INFO @ 21:45:47] [setup] GPU Tracking...
[codecarbon INFO @ 21:45:47] No GPU found.
[codecarbon INFO @ 21:45:47] [setup] CPU Tracking...
[codecarbon WARNING @ 21:45:47] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 21:45:48] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
[codecarbon INFO @ 21:45:48] >>> Tracker's metadata:
[codecarbon INFO @ 21:45:48]   Platform system: Linux-6.5.0-14-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 21:45:48]   Python version: 3.10.9
[codecarbon INFO @ 21:45:48]   CodeCarbon version: 2.3.2
[codecarbon INFO @ 21:45:48]   Available RAM : 15.458 GB
[codecarbon INFO @ 21:45:48]   CPU count: 8
[codecarbon INFO @ 21:45:48]   CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
[codecarbon INFO @ 21:45:48]   GPU count: None
[codecarbon INFO @ 21:45:48]   GPU model: None


Epoch 1/10


2024-01-15 21:45:51.896197: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8522459000 exceeds 10% of free system memory.
2024-01-15 21:45:58.930291: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-15 21:45:58.932211: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-15 21:45:58.933611: I tensorflow/core/common_runtime/executor.cc:1197] [/de

  76/1202 [>.............................] - ETA: 1:18 - loss: 7.5570 - accuracy: 0.0591

[codecarbon INFO @ 21:46:06] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.7967658042907715 W
[codecarbon INFO @ 21:46:06] Energy consumed for all CPUs : 0.000031 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 21:46:06] 0.000055 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 21:59:36] Energy consumed for RAM : 0.001328 kWh. RAM Power : 5.7967658042907715 W
[codecarbon INFO @ 21:59:36] Energy consumed for all CPUs : 0.001718 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 21:59:36] 0.003046 kWh of electricity used since the beginning.


1003/1202 [========================>.....] - ETA: 13s - loss: 5.0208 - accuracy: 0.1468

[codecarbon INFO @ 21:59:51] Energy consumed for RAM : 0.001352 kWh. RAM Power : 5.7967658042907715 W
[codecarbon INFO @ 21:59:51] Energy consumed for all CPUs : 0.001750 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 21:59:51] 0.003102 kWh of electricity used since the beginning.


1202/1202 [==============================] - ETA: 0s - loss: 5.0270 - accuracy: 0.1464
Epoch 10: accuracy improved from 0.14218 to 0.14637, saving model to model/weights-QuijoteNLP.hdf5
1202/1202 [==============================] - 84s 70ms/step - loss: 5.0270 - accuracy: 0.1464


[codecarbon INFO @ 22:00:05] Energy consumed for RAM : 0.001375 kWh. RAM Power : 5.7967658042907715 W
[codecarbon INFO @ 22:00:05] Energy consumed for all CPUs : 0.001779 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 22:00:05] 0.003153 kWh of electricity used since the beginning.


In [ ]:
!carbonboard --filepath="emissions.csv" --port=3333

Dash is running on http://127.0.0.1:3333/

 * Serving Flask app 'codecarbon.viz.carbonboard'
 * Debug mode: off
 * Running on http://127.0.0.1:3333
Press CTRL+C to quit
127.0.0.1 - - [15/Jan/2024 22:01:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 22:01:55] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 22:01:55] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 22:01:55] "GET /_favicon.ico?v=2.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 22:01:55] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2024 22:01:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2024 22:01:55] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2024 22:01:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 22:01:55] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [1

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


## Predicción de un texto nuevo
* Le damos unas palabras para que pueda crear unas lineas contextualizadas en el Quijote.

In [ ]:
seed_text = "leer libros de caballerías"
next_words = 50
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

#### Se guarda la predicción en un archivo de texto

In [ ]:
file = open("prediction.txt", "w") 
file.write(seed_text) 
file.close() 

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [ ]:
#cantidad de palabras unicas
len(tokenizer.word_index.keys())

In [ ]:
#Convertimos el diccionario en una lista, obteniendo solo las palabras
words_list = [(k) for k in tokenizer.word_index.keys()]

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(words_list):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')